In [10]:
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import dgl
from GraphSage import GraphSageNet, GraphSageNet_sampler ,GraphMultiNet, GNN
import time
import os
import wandb
import time
from tqdm import tqdm
import argparse
import yaml
import sys
sys.path.append('../preprocessing')
from UAVidToolKit.colorTransformer import UAVidColorTransformer
from mit_semseg.utils import AverageMeter, colorEncode, accuracy, intersectionAndUnion
import glob
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from skimage.io import imread
import pandas as pd
# import sys
import warnings
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from skimage.segmentation import slic, mark_boundaries
from skimage.io import imsave
import os
import torch
warnings.filterwarnings(action='ignore')



class Tester(object):
    def __init__(self, config):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        config['device'] = self.device
        self.batch_size = config['training']['batch_size']
        self.train_id =  config['test']['train_id']
        
        self.model_path = os.path.join(config['test']['ckpt_dir'] ,'ckpt_'+str(self.train_id),config['test']['ckpt_file'])
        self.save_dir = os.path.join(config['test']['save_result_dir']);   os.makedirs(self.save_dir, exist_ok = True)
        
        self.n_class = config['training']['n_classes']
        self.sampler = config['sampler']['sampler_true']
        self.sampler_neighbor = config['sampler']['sampler_neighbor']
        # self.test_data= data_test_generator(config)
        # multi_scale mode
        self.multi_scale = config['multi_scale_mode']['use_multi_scale']
        if self.sampler : 
            self.model = GraphSageNet_sampler(config).to(self.device)
        elif self.multi_scale : 
            self.model = GraphMultiNet(config).to(self.device)
        # elif config['test_mode']:
        #     self.model = GNN(config).to(self.device)
        else : 
            self.model = GraphSageNet(config).to(self.device)
        # self.model = torch.nn.DataParallel(self.model)
        self.result_name = 'result'+'_'+str(self.train_id)
        
        
    def test(self):
        self.model.load_state_dict(torch.load(self.model_path,map_location=self.device),strict=False)
        self.model.eval()
        torch.cuda.empty_cache()
        print('dataloading...')
        self.raw_data = pd.read_pickle(os.path.join(self.config['data']['pickle_dir'], self.config['test']['test_pickle_name']))  
        with torch.no_grad(): 
            
            
            print('Test...')
            test_acc = []
            save_test_result = []
            
            for idx, row in tqdm(self.raw_data.iterrows()):
                spixel= row['superpixel_segment']
                gt_path = row['gt_path']
                G = row['G']
                feature = row['feature']
                feature[:,:3] = feature[:,:3] / 255 # rgb normalization
                edges = row['edges']
                label_gt = row['label_gt']
                num_nodes = len(label_gt)
                edges_src = torch.tensor(edges[:,0])
                edges_dst = torch.tensor(edges[:,1])
                dgel_graph = dgl.graph((edges_src , edges_dst), num_nodes=num_nodes, idtype=torch.int32)
                dgel_graph.ndata['feat'] = torch.from_numpy(feature)
                dgel_graph.ndata['label'] = torch.from_numpy(np.array(label_gt))
                dgel_graph = dgl.add_self_loop(dgel_graph)
                
                if self.sampler :
                    sampler = dgl.dataloading.MultiLayerNeighborSampler(self.sampler_neighbor)
                    dataloader = dgl.dataloading.NodeDataLoader(
                        dgel_graph, torch.arange(dgel_graph.number_of_nodes()).to(torch.int32), sampler,
                        shuffle=True,drop_last=False)
                    for input_nodes, output_nodes, blocks in dataloader:
                        feature_test  = blocks[0].srcdata['feat'].to(self.device)
                        label_test = blocks[-1].dstdata['label'].to(self.device)
                        # weight = blocks[-1].dstdata['pixel_num'].to(self.device)
                        blocks = [b.to(torch.device(self.device)) for b in blocks]
                        output_test = self.model(blocks, feature_test) 
                        
                elif self.multi_scale : 
                    feature_test  = dgel_graph.ndata['feat'].to(self.device)
                    label_test = dgel_graph.ndata['label'].to(self.device)
                    G_test = dgel_graph.to(self.device)
                    
                    
                    output_test_1, output_test_2, output_test_3 = self.model(G_test, feature_test)
                    output_test = (output_test_1+ output_test_2+ output_test_3) / 3
                    
                    # output_test_1, output_test_2, output_test_3, output_test_4 = self.model(G_test, feature_test)
                    # output_test = (output_test_1+ output_test_2+ output_test_3+output_test_4) / 4
                    
                    # output_test = output_test_1
                else : 
                    feature_test  = dgel_graph.ndata['feat'].to(self.device)
                    label_test = dgel_graph.ndata['label'].to(self.device)
                    G_test = dgel_graph.to(self.device)
                    output_test = self.model(G_test, feature_test) 
                label_test_onehot = F.one_hot(label_test.to(torch.int64), self.n_class) 
                label_test = torch.max(label_test_onehot, 1)[1]
    
                # calculate accuracy
                
                pred = output_test.argmax(dim=1, keepdim=True)
                 
                pred = output_test.argmax(dim=1, keepdim=True)
                crr = pred.eq(label_test.view_as(pred)).sum().item()
                acc = crr / len(pred)
                test_acc.append(acc)
                feature_test = feature_test.cpu().numpy()
                label_test = label_test.cpu().numpy()
                
                # output_test = output_test.cpu().numpy()
                # label_test = label_test.cpu().numpy()
                pred = pred.cpu().numpy()
                
                # save_test_result.append([spixel, gt_path, feature_test,label_test,G,output_test,label_gt,pred])
                save_test_result.append([spixel, gt_path, label_test,pred])
            
            print("validation acc : {:.6f}".format( np.mean(test_acc) ))
            pred = pd.DataFrame(save_test_result)
            # df.columns = ['spixel','gt_path','feature_test','label_test','G','output_test','label_gt','pred']
            pred.columns = ['spixel','gt_path','label_test','pred']
            
            print('Save Dataframe...')
            pred.to_pickle(os.path.join(self.save_dir,self.result_name + '.pickle'))
            
            print("Start Evaluation....")
            
            #### eval part
            # num = pred_pickle_path.split('/')[1].split('_')[1].split('.')[0]
            save_path = f'convert_test_set/pred_g2i_{self.train_id}/'
            os.makedirs(save_path,exist_ok=True)
            convert_Graph_to_Image_function(save_path, pred,self.device)
            pred_path = save_path
            
            
            return pred_path
    
def convert_Graph_to_Image_function(save_path, df,device):
    
    for px in tqdm(range(len(df))):
        
        row = df.iloc[px]
        name_path = row['gt_path']
        name = name_path.split('/')
        
        save_name_pred = save_path + name[4] + '_' + name[-1].split('.')[0]
        # print(save_name_pred)
        
        G2_img = np.zeros((row['spixel'].shape[0], row['spixel'].shape[1]))
        

        G2_img = torch.from_numpy(G2_img)
        G2_img = G2_img.to(torch.device(device))
        
        prediction_cl = np.array(row['pred'])
        test_cl = np.array(row['label_test'])
        superpixel_img = np.array(row['spixel'])
        
        node_num = np.amax(superpixel_img)
                           
        prediction_cl = torch.from_numpy(prediction_cl)
        prediction_cl = prediction_cl.to(torch.device(device))
        
        test_cl = torch.from_numpy(test_cl)
        test_cl = test_cl.to(torch.device(device))
    
        superpixel_img = torch.from_numpy(superpixel_img)
        superpixel_img = superpixel_img.to(torch.device(device))

        for ix in range(node_num):
            
            superpixel_cluster = (superpixel_img[:,:] == ix)
            
            
            prediction_superpixel_cluster_node_mapping = torch.multiply(superpixel_cluster, prediction_cl[ix])
            
            
            
            G2_img = torch.add(G2_img, prediction_superpixel_cluster_node_mapping)
            
        
        G2_img = G2_img.cpu().numpy()
        
        
        np.save(save_name_pred, G2_img)
        # imsave(save_name_pred, G2_img)
        
    

def parse(path):
    with open(path, 'r') as f:
        config = yaml.safe_load(f)
        f.close()
    return config

In [ ]:
now = int(time.time())
# parser = argparse.ArgumentParser()
# parser.add_argument('--config_yaml','-yml', type=str, default='train_hi.yaml')
# args = parser.parse_args()
config_path = f'yml/train_uav_multi_SY.yaml'
# config_path = f'yml/train_uav_test.yaml'
# config_path = f'yml/train_uav.yaml'
config = parse(config_path)
# os.environ['CUDA_VISIBLE_DEVICES']=str(config['training']['gpu']['id'] )    
os.environ['CUDA_VISIBLE_DEVICES']='0'


wandb_name = config['test']['wandb_name']
try :
    wandb_names = wandb_name.split('_')
    config['hidden_dim'] = int(wandb_names[4])
    config['out_dim'] = int(wandb_names[5])
    config['Layer'] = int(wandb_names[6])
    config['test']['train_id'] = wandb_names[-1]
    print(f"train id : {config['test']['train_id']}, hidden dim : {config['hidden_dim']}, out_dim : {config['out_dim']} , Layers : {config['Layer']}")
except : 

    print(f"train id : {config['test']['train_id']}")

tester = Tester(config=config)
pred_path = tester.test()

train id : 1639144604, hidden dim : 256, out_dim : 256 , Layers : 10
dataloading...
Test...


70it [00:02, 30.65it/s]


validation acc : 0.692261
Save Dataframe...
Start Evaluation....


 81%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                           | 57/70 [14:43<03:33, 16.42s/it]

In [ ]:
def evaluation(pred_path,n_class):
    pred_list = glob.glob(pred_path+'/*')
    count = 0

    jaccard_list = []
    for ix in tqdm(range(len(pred_list))):
        pred_name = pred_list[ix]

        
        # uavid_patch
        # pred_seq = pred_name.split('/')[-1].split('_')[0]
        # pred_patch = pred_name.split('/')[-1].split('_')[1]
        # pred_num = pred_name.split('/')[-1].split('_')[2]
        # gt_path = '../data/uavid/uavid_val_patch(2048)/'+ pred_seq + '/Labels/' + pred_patch+'_'+pred_num
        # gt_img = imread(gt_path.replace('npy','png'))
        
        # uavid_full
        pred_seq = pred_name.split('/')[-1].split('_')[0]
        pred_num = pred_name.split('/')[-1].split('_')[1]
        gt_path = '../data/uavid/uavid_val/'+ pred_seq + '/Labels/' +pred_num
        gt_img = imread(gt_path.replace('npy','png'))
        clrEnc = UAVidColorTransformer()
        gt_img = clrEnc.transform(gt_img, dtype=np.uint8)

        #city 
        # city = pred_name.split('/')[-1].split('_')[1]
        # label_name = pred_name.split('/')[-1][4:-4:1]
        # gt_img = imread(f'../data/cityscape/gtFine/val/{city}/{label_name}')
        # gt_img = np.vectorize(mapping_20.get)(gt_img)

        pred = np.load(pred_name)
        intersection , union = intersectionAndUnion(pred, gt_img, n_class)
        jaccard_list.append(intersection/union)
    df = pd.DataFrame(jaccard_list)
    # df.columns=['Clutter','Building','Road','Static_Car','Tree','Vegetation','Human','Moving_Car']
    print('Saving Dataframe....')
    # df.to_csv(f'result_1')
    describe=df.describe()
    print("mIoU : ", describe.mean(axis=1)['mean'] * 100)
    print('Done!')
    return describe, pred_list

In [ ]:
# train_id =str(1638019938)
# pred_path = f'convert_test_set/pred_g2i_{train_id}_full/'
# train_id =str(1638155754)
# pred_path = f'convert_test_set/pred_g2i_{train_id}/'

describe, pred_list = evaluation(pred_path,8)

In [ ]:
describe.columns=['Clutter','Building','Road','Static_Car','Tree','Vegetation','Human','Moving_Car']
describe


In [ ]:
describe.to_csv(f'csv_result/result_{wandb_name}.csv')

In [ ]:
import matplotlib.pyplot as plt
from skimage.io import imshow

from random import *
 
i = randint(1, len(pred_list)) 
# i = 247
fig = plt.figure(figsize=(20,10))
ax0 = fig.add_subplot(1,3,1)
ax1 = fig.add_subplot(1,3,2)
ax2 = fig.add_subplot(1,3,3)
pred_img_path = pred_list[i]

clrEnc = UAVidColorTransformer()
pred_img = np.load(pred_img_path)

# uavid patch
# pred_seq = pred_img_path.split('/')[-1].split('_')[0]
# pred_patch = pred_img_path.split('/')[-1].split('_')[1]
# pred_num = pred_img_path.split('/')[-1].split('_')[2]
# gt_path = '../data/uavid/uavid_val_patch(1024)/'+ pred_seq + '/Labels/' + pred_patch+'_'+pred_num
# gt_img = imread(gt_path.replace('npy','png'))

# uavid full
pred_seq = pred_img_path.split('/')[-1].split('_')[0]
pred_num = pred_img_path.split('/')[-1].split('_')[1]
gt_path = '../data/uavid/uavid_val/'+ pred_seq + '/Labels/' + pred_num
gt_img = imread(gt_path.replace('npy','png'))
gt_img = clrEnc.transform(gt_img, dtype=np.uint8)



intersection , union = intersectionAndUnion(pred_img, gt_img, 8)
print(gt_img.shape)
gt_img = clrEnc.inverse_transform(gt_img)
pred_img = clrEnc.inverse_transform(pred_img)
# print(['Clutter','Building','Road','Static_Car','Tree','Vegetation','Human','Moving_Car'])
# print('')
# print(intersection)
# print('')
# print(union)
# print('')
# print(intersection/union)

ax1.imshow(gt_img)
ax2.imshow(pred_img)
img_path = gt_path.replace('Labels','Images').replace('npy','png')
img = imread(img_path)
ax0.imshow(img)
plt.show()

In [8]:
import os
os.listdir('./checkpoints/ckpt_1639017980')

['epoch_032_ckpt.pth',
 'epoch_005_ckpt.pth',
 'epoch_041_ckpt.pth',
 'epoch_018_ckpt.pth',
 'epoch_013_ckpt.pth',
 'epoch_035_ckpt.pth',
 'epoch_006_ckpt.pth',
 'epoch_049_ckpt.pth',
 'epoch_044_ckpt.pth',
 'epoch_002_ckpt.pth',
 'epoch_019_ckpt.pth',
 'epoch_014_ckpt.pth',
 'epoch_015_ckpt.pth',
 'epoch_025_ckpt.pth',
 'epoch_046_ckpt.pth',
 'epoch_036_ckpt.pth',
 'epoch_048_ckpt.pth',
 'epoch_007_ckpt.pth',
 'epoch_024_ckpt.pth',
 'epoch_008_ckpt.pth']

In [9]:
!nvidia-smi

Sun Dec 12 00:18:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 465.27       CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 35%   39C    P8    29W / 350W |  14779MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:5E:00.0 Off |                  N/A |
| 43%   